**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
!pip install scikit-video
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization



Using TensorFlow backend.


    100% |████████████████████████████████| 2.3MB 12.8MB/s 


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            #removed because of poor performance
            #a = self.learned_act(s)
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act allows us to pick up an action based on an $\epsilon$-greedy policy: with probability $\epsilon$, we decide to choose a random action, and with probabilty $1-\epsilon$, we decide to choose an action according to the policy that we previously learned. This parameter is essential because it controls how much we want to explore. Indeed, one can imagine that only playing according to the information we previously gather while playing will prevent from learning new and  crucial information to improve our learned policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [47]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path_to_data = '/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/'

class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/' + str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1 
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

Mounted at /content/drive


The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=275# set small when debugging
epochs_test=50 #set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array tells us where the mouse can go or not (0 if she can, -1 otherwise). The board array contains in position x,y the possible reward: +0.5 (cheese), -1 (poison) or 0 (nothing).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0, self.n_action)
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        #reseting the env
        previous_state = env.reset()
        game_over = True
        win = 0
        lose = 0
        while(game_over):
          action = agent.act(previous_state)
          #getting the next, reward and game over
          new_state, reward, new_game_over = env.act(action)
          previous_state = new_state
          
          if reward >0:
            win += reward
          else:
            lose -= reward
          game_over = not new_game_over
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [18]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/' + 'random0.mp4'))

Win/lose count 9.5/12.0. Average score (-2.5)
Win/lose count 12.5/11.0. Average score (-0.5)
Win/lose count 9.5/8.0. Average score (0.16666666666666666)
Win/lose count 12.5/21.0. Average score (-2.0)
Win/lose count 8.5/12.0. Average score (-2.3)
Win/lose count 8.5/21.0. Average score (-4.0)
Win/lose count 9.0/14.0. Average score (-4.142857142857143)
Win/lose count 8.5/14.0. Average score (-4.3125)
Win/lose count 10.0/13.0. Average score (-4.166666666666667)
Win/lose count 10.5/8.0. Average score (-3.5)
Final score: -3.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We have that : 
\begin{align} Q^\pi(s,a) &= \mathbb{E}_{p^{\pi}}\left[\sum_t \gamma^{t}r(s_t,a_t)|s_0=s,a_0=a\right] =& 
r(a, s) + \gamma \mathbb{E}_{p^{\pi}}\left[\sum_{t \ge 1} \gamma^{t - 1} r(s_t,a_t)|s_0=s,a_0=a\right] \end{align}
Then, we have:
 \begin{align}Q^\pi(s,a) &= r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'| s_0 = s) \mathbb{E}_{p^{\pi}}\left[\sum_{t \ge 1} \gamma^{t - 1} r(s_t,a_t)|s_1=s',a_1=\pi(s)\right] 
\end{align}
 And then
 \begin{align}
Q^\pi(s,a) =  E_{(s',a')\sim p(.|s,a)}\left[r(s,a)+\gamma Q^{\pi}(s',a')\right] \end{align}

Finally, we have  \begin{align} Q^{*}(s, a) &= \max_{\pi} Q^{\pi}(s, a) \ &= \max_{\pi} \left[ r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s' | s_0 = s) Q^\pi(s', a = \pi(s)) \right] = r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'| s_0 = s,) \max_{\pi} Q^\pi(s',\pi(s)) \end{align}

Hence, 
\begin{align}
 Q^{*}(s, a) =r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s'| s_0 = s)max_{a'}Q^{*}(s', a')  
\end{align}

We can then conclude that : 
\begin{align} & Q^{*}(s, a) = \mathbb{E}_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')\right] 
\end{align}

From that, we have :
\begin{align}
 0= \mathbb{E}_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma\max_{a'}Q^{*}(s',a') - Q^{*}(s, a) \right] 
\end{align}

Thus, a plausible objective solution is to minimze the variance of the random variable, that is :
\begin{equation*}
\mathcal{L}(\theta)=\mathbb{E}_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        #we change our stored data if we have too much of them
        if len(self.memory) > self.max_memory:
          self.memory = self.memory[1:]

    def random_access(self):
        index = np.random.randint(len(self.memory))
        information = self.memory[index]
        return information

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            #saving the video
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights= prefix+'model.h5',
                   name_model= prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
from keras.layers import Flatten

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s_res = s.reshape(-1, 5, 5, self.n_state)
        q = self.model.predict(s_res)
        action = np.argmax(q)
        return action

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        next_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        #retrieving from memory instances to store state and next state
        instances = [self.memory.random_access() for i in range(self.batch_size)]
        for i in range(self.batch_size):
            previous_state, next_state, action, reward, game_over = instances[i]
            input_states[i] = previous_state
            next_states[i] = next_state
            
        # computing q action value function
        target_q = self.model.predict(input_states)
        q_next = self.model.predict(next_states)
        
        #creating the Q function we want to learn
        for i in range(self.batch_size):
            state, next_state, action, reward, game_over = instances[i]
            
            if game_over_:
                target_q[i, action] += reward
            else:
                target_q[i, action] += reward + self.discount * np.max(q_next[i])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5', name_model='model.json'):
        self.model.save_weights('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/' + 
                                name_weights, overwrite=True)
        name_model = '/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/' + name_model
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',
             name_model='model.json'):
        name_model = '/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/' + name_model
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/' + name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(15, activation='relu'))
        model.add(Dense(4))

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [49]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/fc_train0.mp4'))

Epoch 000/275 | Loss 0.0014 | Win/lose count 1.5/5.0 (-3.5)
Epoch 001/275 | Loss 0.0086 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/275 | Loss 0.0041 | Win/lose count 3.5/8.0 (-4.5)
Epoch 003/275 | Loss 0.0143 | Win/lose count 3.5/5.0 (-1.5)
Epoch 004/275 | Loss 0.0093 | Win/lose count 3.5/4.0 (-0.5)
Epoch 005/275 | Loss 0.0111 | Win/lose count 2.5/3.0 (-0.5)
Epoch 006/275 | Loss 0.0012 | Win/lose count 2.0/3.0 (-1.0)
Epoch 007/275 | Loss 0.0084 | Win/lose count 3.5/3.0 (0.5)
Epoch 008/275 | Loss 0.0006 | Win/lose count 3.5/3.0 (0.5)
Epoch 009/275 | Loss 0.0008 | Win/lose count 3.0/10.0 (-7.0)
Epoch 010/275 | Loss 0.0005 | Win/lose count 1.0/4.0 (-3.0)
Epoch 011/275 | Loss 0.0323 | Win/lose count 3.5/6.0 (-2.5)
Epoch 012/275 | Loss 0.0081 | Win/lose count 1.5/0 (1.5)
Epoch 013/275 | Loss 0.0083 | Win/lose count 3.5/6.0 (-2.5)
Epoch 014/275 | Loss 0.0164 | Win/lose count 2.5/8.0 (-5.5)
Epoch 015/275 | Loss 0.0085 | Win/lose count 1.5/6.0 (-4.5)
Epoch 016/275 | Loss 0.0004 | Win/lose count

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # Convolutional layers
        
        model = Sequential()
       
        model.add(Conv2D(16, (3,3), activation='relu'))
        model.add(Conv2D(8, (3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [45]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/cnn_train0.mp4'))

Epoch 000/275 | Loss 0.0044 | Win/lose count 8.5/15.0 (-6.5)
Epoch 001/275 | Loss 0.0063 | Win/lose count 6.5/10.0 (-3.5)
Epoch 002/275 | Loss 0.0043 | Win/lose count 1.5/6.0 (-4.5)
Epoch 003/275 | Loss 0.0100 | Win/lose count 9.0/6.0 (3.0)
Epoch 004/275 | Loss 0.0022 | Win/lose count 4.5/3.0 (1.5)
Epoch 005/275 | Loss 0.0022 | Win/lose count 6.5/6.0 (0.5)
Epoch 006/275 | Loss 0.0041 | Win/lose count 9.5/7.0 (2.5)
Epoch 007/275 | Loss 0.0041 | Win/lose count 6.5/5.0 (1.5)
Epoch 008/275 | Loss 0.0040 | Win/lose count 4.0/6.0 (-2.0)
Epoch 009/275 | Loss 0.0022 | Win/lose count 2.5/10.0 (-7.5)
Epoch 010/275 | Loss 0.0041 | Win/lose count 4.5/4.0 (0.5)
Epoch 011/275 | Loss 0.0079 | Win/lose count 7.0/7.0 (0.0)
Epoch 012/275 | Loss 0.0022 | Win/lose count 5.0/12.0 (-7.0)
Epoch 013/275 | Loss 0.0041 | Win/lose count 8.5/12.0 (-3.5)
Epoch 014/275 | Loss 0.0060 | Win/lose count 8.0/13.0 (-5.0)
Epoch 015/275 | Loss 0.0040 | Win/lose count 7.5/11.0 (-3.5)
Epoch 016/275 | Loss 0.0059 | Win/lose c

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [57]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/3.0. Average score (-2.0)
Win/lose count 2.0/3.0. Average score (-1.5)
Win/lose count 4.0/6.0. Average score (-1.6666666666666667)
Win/lose count 4.5/3.0. Average score (-0.875)
Win/lose count 0.5/5.0. Average score (-1.6)
Win/lose count 2.5/3.0. Average score (-1.4166666666666667)
Win/lose count 1.0/6.0. Average score (-1.9285714285714286)
Win/lose count 2.5/4.0. Average score (-1.875)
Win/lose count 4.5/5.0. Average score (-1.7222222222222223)
Win/lose count 2.0/1.0. Average score (-1.45)
Win/lose count 1.5/1.0. Average score (-1.2727272727272727)
Win/lose count 4.0/1.0. Average score (-0.9166666666666666)
Win/lose count 5.0/1.0. Average score (-0.5384615384615384)
Win/lose count 2.5/5.0. Average score (-0.6785714285714286)
Win/lose count 3.5/3.0. Average score (-0.6)
Win/lose count 2.0/3.0. Average score (-0.625)
Win/lose count 2.5/6.0. Average score (-0.7941176470588235)
Win/lose count 2.5/3.0. Average score (-0.7777777777777778)
Win/lose count 7.

In [42]:
HTML(display_videos('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/cnn_test8.mp4'))

In [54]:
HTML(display_videos('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/fc_test48.mp4'))

The CNN agent seems to be slightly better than the FC agent. 
For both of the agent, we can see a problem of exploration for the mouse: after having retrieving the cheese on a zone of the map, the mouse has a tendancy to stay and go back and forth in this area, instead of going further and explore other areas in the map.
Increasing the temperature from 0.3 or decreasing it from 0.3 seems to increase  the score on average for both the agent.
Maybe the 0.3 value is a 'sort of equilibrium value', where it is more difficult for the agent to learn a balanced policy to retrieve cheese and avoid poisonus cell.  

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [70]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train = True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            #saving the video
            env.draw(prefix+str(e))
            agent.set_epsilon(agent.epsilon * 0.8)

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights= prefix+'model.h5',
                   name_model= prefix+'model.json')
        

    
    
    
    
    
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/' + str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1 
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        #adding the malus
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        reward += self.board[self.x, self.y] 
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        #state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                       #self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        #state = np.concatenate((
                               #self.board.reshape(self.grid_size, self.grid_size,1),
                        #self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

    
    
    
"""    
    ## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)
## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
"""

'    \n    ## use those samples of code:\n#In train explore:\nstate, reward, game_over = env.act(action, train=True)\n## In Environment exploring:\n# You will have to change n_state to 3 because you will use one more layer!\nreward = 0\nif train:\n    reward = -self.malus_position[self.x, self.y]\nself.malus_position[self.x, self.y] = 0.1\n\nreward = reward + self.board[self.x, self.y]\n# 3 "feature" states instead of 2\nstate = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),\n                                self.board.reshape(self.grid_size, self.grid_size,1),\n                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)\n'

In [80]:
# Training
epoch_train = 250
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/cnn_train_explore10.mp4'))

Epoch 000/275 | Loss 0.0029 | Win/lose count 9.0/21.400000000000052 (-12.400000000000052)
Epoch 001/275 | Loss 0.0273 | Win/lose count 5.8/30.500000000000103 (-24.700000000000102)
Epoch 002/275 | Loss 0.0267 | Win/lose count 5.800000000000001/30.50000000000013 (-24.70000000000013)
Epoch 003/275 | Loss 0.0441 | Win/lose count 9.600000000000003/36.40000000000015 (-26.800000000000146)
Epoch 004/275 | Loss 0.0192 | Win/lose count 4.4/35.50000000000013 (-31.10000000000013)
Epoch 005/275 | Loss 0.0537 | Win/lose count 9.100000000000003/29.00000000000012 (-19.90000000000012)
Epoch 006/275 | Loss 0.0118 | Win/lose count 6.400000000000001/32.40000000000013 (-26.000000000000124)
Epoch 007/275 | Loss 0.0195 | Win/lose count 9.800000000000002/32.00000000000009 (-22.200000000000088)
Epoch 008/275 | Loss 0.0115 | Win/lose count 7.200000000000002/38.30000000000019 (-31.100000000000186)
Epoch 009/275 | Loss 0.0101 | Win/lose count 3.9999999999999996/27.100000000000083 (-23.100000000000083)
Epoch 010/2

In [76]:
# Evaluation
#agent.epsilon
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('/content/drive/My Drive/DEEP LEARNING VINCENT LEPETIT/cnn_test_explore8.mp4'))

Win/lose count 0.4/20.000000000000014. Average score (-19.600000000000016)
Win/lose count 0/21.10000000000003. Average score (-20.350000000000023)
Win/lose count 0.4/21.90000000000004. Average score (-20.733333333333363)
Win/lose count 1.2000000000000002/20.80000000000003. Average score (-20.450000000000028)
Win/lose count 0/22.100000000000044. Average score (-20.78000000000003)
Win/lose count 0/20.100000000000016. Average score (-20.666666666666696)
Win/lose count 0.8/20.90000000000003. Average score (-20.585714285714314)
Win/lose count 0/22.100000000000044. Average score (-20.77500000000003)
Win/lose count 0.4/22.000000000000043. Average score (-20.8666666666667)
Win/lose count 0.4/20.000000000000014. Average score (-20.74000000000003)
Win/lose count 0.4/21.00000000000003. Average score (-20.72727272727276)
Win/lose count 0.8/20.90000000000003. Average score (-20.67500000000003)
Win/lose count 0.4/21.000000000000032. Average score (-20.6692307692308)
Win/lose count 0/23.1000000000000

We do not see any progression while using a exploring policy: maybe it is a bug of my, or maybe during the training, the q function does not learn to avoid previously visited cell.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***